# Incorporating Iowa educational data

### Import relevant libraries

In [1]:
import pandas as pd

### Import English and Math proficiency rates for grades 3-8 and 10-11

In [2]:
from read_path_module import read_data_relative_path # home grown function to read data above current directory

ELA_IEP = read_data_relative_path(relative_dataset_path = './data/educate_iowa/2018-2019 ISASP Proficiency Rates by District and Grade_final_simplified.xlsx',
                           data_type='excel',
                           sheet='ELA'
                          )
Math_IEP = read_data_relative_path(relative_dataset_path = './data/educate_iowa/2018-2019 ISASP Proficiency Rates by District and Grade_final_simplified.xlsx',
                           data_type='excel',
                           sheet='Math'
                          )

### Import zip code and county lookup table for each of neighborhoods

In [3]:
Hood_Lookup = read_data_relative_path(relative_dataset_path = './data/educate_iowa/created/Neighborhoods_Zip_Code_Lookup.xlsx',
                                data_type='excel')

### Filter each dataframe down to County Name and % Proficient

In [4]:
# Create list of Proficiency % column names
cols_Proficiency_IEP = [col for col in ELA_IEP.columns if '% Proficient' in col]

# Filter down to Ames School District and Select only columns of interest for ELA scores
ELA_IEP = ELA_IEP.loc[ELA_IEP['District Name'] == 'Ames Comm School District']
ELA_IEP = ELA_IEP[['County Name'] + cols_Proficiency_IEP]
ELA_IEP = ELA_IEP.add_prefix('ELA ')

# Filter down to Ames School District and Select only columns of interest for math scores
Math_IEP = Math_IEP.loc[Math_IEP['District Name'] == 'Ames Comm School District']
Math_IEP = Math_IEP[['County Name'] + cols_Proficiency_IEP]
Math_IEP = Math_IEP.add_prefix('Math ')

### Merge two tables so that we have proficiency by grade by county

In [12]:
education_by_county_ELA = pd.merge(Hood_Lookup, ELA_IEP, how='left', left_on='County', right_on='ELA County Name')
education_by_county = pd.merge(education_by_county_ELA, Math_IEP, how='left', left_on='County', right_on='Math County Name')

### Export to csv

In [13]:
# Create list of Proficiency % column names
cols_Proficiency_IEP = [col for col in education_by_county.columns if '% Proficient' in col]

# Average math and ela percentages
education_by_county['Avg_Proficiency'] = education_by_county[cols_Proficiency_IEP].mean(axis=1)

# Drop column name
education_by_county = education_by_county.drop(['Source', 'ELA County Name', 'Math County Name'] + cols_Proficiency_IEP, axis=1)

# Export to csv
# education_by_county.to_csv('/Users/michaellink/Desktop/__NYCDSA/_Projects/Machine_Learning/data/educate_iowa/created/education_by_county.csv',
#                           index = False)

from write_path_module import write_data_relative_path

write_data_relative_path(df = education_by_county,
                         relative_dataset_path = './data/educate_iowa/created/education_by_county.csv'
                        )